In [8]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
years = pd.read_csv("/opt/ml/input/data/train/years.tsv", delimiter="\t")
uim = pd.read_csv("/opt/ml/input/workspace/Item_Removal/new_user_movie_interaction.csv")
raw_data = pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")

In [57]:
group = raw_data.groupby("user")["time"].apply(max)
group = group.apply(dt.datetime.fromtimestamp)
for user in group.keys() :
    group[user] = group[user].year + 1
group.to_csv("user_year.csv")

In [62]:
user_year = pd.read_csv("user_year.csv")
user_year_np = user_year["time"].to_numpy()

In [96]:
unique_sid = pd.read_csv("unique_sid.txt", header=None)
unique_sid_list = list()
for sid in unique_sid[0] :
    unique_sid_list.append(sid)
id2show = dict((i, sid) for (i, sid) in enumerate(unique_sid_list))
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid_list))

In [107]:
for key in years["item"].keys() :
    years["item"][key] = show2id[years["item"][key]]

In [154]:
id2year = dict()
for i in range(len(years["item"])) :
    id2year[years["item"][i]] = years["year"][i]
missing_years = list()

for i in range(6707) :
    if i not in id2year.keys() :
        missing_years.append(i)

In [123]:
uim_np = uim.iloc[:,1:].to_numpy()

In [156]:
# 3분 정도 소요
for user in range(len(uim_np)) :
    for item in range(len(uim_np[user])) :
        
        if item in missing_years :
            continue

        item_year = id2year[item]
        if id2year[item] >= user_year_np[user] :
            uim_np[user][item] = 2

    if user%5000 == 0 :
        print(f"{user}/31360 users completed")

0/31360 users completed
5000/31360 users completed
10000/31360 users completed
15000/31360 users completed
20000/31360 users completed
25000/31360 users completed
30000/31360 users completed


In [159]:
new_uim_df = pd.DataFrame(uim_np)
#3분 정도 소요
new_uim_df.to_csv("future_removed_uim.csv")